# install

In [ ]:
%pip install -U torch torchaudio torchdata torchtext torchvision torchinfo

In [3]:
!pip install tiktoken
!pip install modelscope
!pip install transformers_stream_generator==0.0.4
!pip install optimum==1.12.0
!pip install auto-gptq
# %pip install transformers_stream_generator==0.0.4
# %pip install modelscope   #>=1.9.0
# %pip install auto-gptq  #==0.4.2
# %pip install optimum==1.12.0

zsh:1: 1.9.0 not found


Note: you may need to restart the kernel to use updated packages.


In [ ]:
!git clone -b v1.0.8 https://github.com/Dao-AILab/flash-attention
!cd flash-attention && pip install .
# 下方安装可选，安装可能比较缓慢。
# Below are optional. Installing them might be slow.
# pip install csrc/layer_norm
# pip install csrc/rotary

In [4]:
import torch
print(torch.__version__)

2.1.0


# model usage

In [1]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
from modelscope import GenerationConfig

model_dir = snapshot_download('TongyiFinance/Tongyi-Finance-14B-Chat-Int4')

# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

2023-11-24 18:05:39,577 - modelscope - INFO - PyTorch version 2.1.0 Found.
2023-11-24 18:05:39,578 - modelscope - INFO - Loading ast index from /Users/jxy/.cache/modelscope/ast_indexer
2023-11-24 18:05:39,677 - modelscope - INFO - No valid ast index found from /Users/jxy/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2023-11-24 18:05:39,711 - modelscope - INFO - Loading done! Current index file version is 1.9.5, with md5 a814d16a7ee2539750fe2738e7c8eb07 and a total number of 945 components indexed
/Users/jxy/miniconda3/envs/stock/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 8.21k/8.21k [00:00<00:00, 2.20MB/s]
Downloading: 100%|██████████| 50.8k/50.8k [00:00<00:00, 159kB/s]
Downloading: 100%|██████████| 1.44k/1.44k [00:00<00:00, 778kB/s]
Downloading: 10

In [7]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
from modelscope import GenerationConfig

model_dir = snapshot_download('TongyiFinance/Tongyi-Finance-14B-Chat-Int4')

# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

# use bf16
# model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", trust_remote_code=True, bf16=True).eval()
# use cpu only
# model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cpu", trust_remote_code=True).eval()
# use auto mode, automatically select precision based on the device.
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cpu", trust_remote_code=True).eval()
# 模型加载指定device_map='cuda:0'，更改成device_map='auto'会使用所有可用显卡

# Specify hyperparameters for generation
model.generation_config = GenerationConfig.from_pretrained(model_dir, trust_remote_code=True)

response, history = model.chat(tokenizer, "请解释一下资产负债率", history=None)
print(response)
# 资产负债率是一个财务比率，用来衡量一个企业的负债水平。它是用一个企业负债总额除以其资产总额的百分比来表示的。它的计算公式是：资产负债率 = 负债总额 / 资产总额。它能够反映一个企业的财务状况，以及它是否具有足够的资产来抵偿其债务。

Detected the presence of a `quantization_config` attribute in the model's configuration but you don't have the correct `bitsandbytes` version to support int8 serialization. Please install the latest version of `bitsandbytes` with  `pip install --upgrade bitsandbytes`.


In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
from modelscope import GenerationConfig

model_dir = snapshot_download('TongyiFinance/Tongyi-Finance-14B')

# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

# use bf16
# model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cuda:0", trust_remote_code=True, bf16=True).eval()
# use cpu only
# model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cpu", trust_remote_code=True).eval()
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cuda:0", trust_remote_code=True).eval()
# 模型加载指定device_map='cuda:0'，更改成device_map='auto'会使用所有可用显卡

# Specify hyperparameters for generation
model.generation_config = GenerationConfig.from_pretrained(model_dir, trust_remote_code=True)

inputs = tokenizer('市盈率是最常用来评估股价水平是否合理的指标之一，是指', return_tensors='pt')
inputs = inputs.to(model.device)
pred = model.generate(**inputs)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))
# 市盈率是最常用来评估股价水平是否合理的指标之一，是指股票价格与每股盈利的比率。...